<h1> Segmenting and Clustering Neighborhoods in Toronto </h1>

In [1]:
import pandas as pd
import numpy as np
!conda install -c conda-forge lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxml2-2.9.10             |       he19cac6_1         1.2 MB
    libxslt-1.1.34             |       hc22bd24_0         432 KB
    lxml-4.5.2                 |   py36h17c4326_0         1.3 MB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.1 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.34-hc22bd24_0

In [2]:
#Getting data from the webpage to dataframe
df_toronto = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df_toronto = df_toronto.rename(columns={'Neighbourhood':'Neighborhood'})
df_toronto.shape

(180, 3)

In [3]:
#dropped the rows with a borough that is Not assigned
df_toronto = df_toronto[~df_toronto['Borough'].isin(['Not assigned'])]
df_toronto.reset_index(drop=True)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
df_toronto.shape

(103, 3)

In [5]:
#to check if neighborhood column has any not assigned neighborhood
if df_toronto['Neighborhood'].str.contains('Not assigned').any():
    print("Not Assigned Neighborhood found")

In [6]:
#getting the latitude longitude data from the provided CSV file
link= 'https://cocl.us/Geospatial_data'
df_latlong = pd.read_csv(link)
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
#Merging the two dataframes to get the final dataframe with latitude and longitude for each postal code
df_torfin= pd.merge(df_toronto, df_latlong, left_on='Postal Code', right_on='Postal Code')
df_torfin.head(10)
print(df_torfin.shape)
len(df_torfin.Neighborhood.unique())

(103, 5)


99

<h4> Creating the url and getting the relevant information using foursquares</h4>

In [8]:
import requests 
from pandas.io.json import json_normalize

In [9]:
CLIENT_ID = 'D502JG0ELHOWV5WZ2ECT32GRHHNS4SLYKPMRRV52G0EFNQNJ' 
CLIENT_SECRET = 'ECYPGKMQ3C400H2FC3T2I40JBYSWH3DXSAVNCZQ04ZQLXJPL' 
VERSION = '20180605' 
LIMIT = 50
radius = 500

In [10]:


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        print(name + ': ' + str(len(results)))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=df_torfin['Neighborhood'],
                                   latitudes=df_torfin['Latitude'],
                                   longitudes=df_torfin['Longitude']
                                  )

toronto_venues.shape

Parkwoods: 4
Victoria Village: 5
Regent Park, Harbourfront: 43
Lawrence Manor, Lawrence Heights: 13
Queen's Park, Ontario Provincial Government: 34
Islington Avenue, Humber Valley Village: 1
Malvern, Rouge: 1
Don Mills: 4
Parkview Hill, Woodbine Gardens: 10
Garden District, Ryerson: 50
Glencairn: 5
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale: 0
Rouge Hill, Port Union, Highland Creek: 3
Don Mills: 21
Woodbine Heights: 5
St. James Town: 50
Humewood-Cedarvale: 4
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood: 8
Guildwood, Morningside, West Hill: 9
The Beaches: 4
Berczy Park: 50
Caledonia-Fairbanks: 4
Woburn: 5
Leaside: 32
Central Bay Street: 50
Christie: 17
Cedarbrae: 8
Hillcrest Village: 4
Bathurst Manor, Wilson Heights, Downsview North: 20
Thorncliffe Park: 21
Richmond, Adelaide, King: 50
Dufferin, Dovercourt Village: 18
Scarborough Village: 2
Fairview, Henry Farm, Oriole: 50
Northwood Park, York University: 4
East Toronto, Broadview North (Old 

(1669, 7)

In [12]:
print(toronto_venues.shape)
toronto_venues.head(10)

(1669, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
5,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
7,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
8,Victoria Village,43.725882,-79.315572,Cash Money,43.725486,-79.312665,Financial or Legal Service
9,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery


<h4> One hot encoding and grouping the neighborhoods </h4>

In [13]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
toronto_onehot.shape

(1669, 249)

In [15]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(97, 249)


,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h4> Calculating the frequnecy of each venue category and finding the top 5 most common venues of each neighborhood</h4>

In [16]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print(hood)
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Agincourt
                       venue  freq
0  Latin American Restaurant   0.2
1               Skating Rink   0.2
2                     Lounge   0.2
3             Breakfast Spot   0.2
4             Clothing Store   0.2


Alderwood, Long Branch
            venue  freq
0     Pizza Place  0.25
1    Skating Rink  0.12
2  Sandwich Place  0.12
3             Gym  0.12
4     Coffee Shop  0.12


Bathurst Manor, Wilson Heights, Downsview North
                 venue  freq
0          Coffee Shop  0.10
1                 Bank  0.10
2             Pharmacy  0.05
3  Fried Chicken Joint  0.05
4          Pizza Place  0.05


Bayview Village
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Mobile Phone Shop  0.00


Bedford Park, Lawrence Manor East
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2          Restaurant  0.09
3      Sandwich Place  0.09
4         Pizza Pla

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Lounge
1,"Alderwood, Long Branch",Pizza Place,Skating Rink,Sandwich Place,Pub,Pool
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sushi Restaurant,Park,Mobile Phone Shop
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Sushi Restaurant


<h2> CLUSTERING </h2>

In [19]:
from sklearn.cluster import KMeans

In [20]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 1, 2, 0, 3, 1, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 2, 3, 3, 0, 3, 3, 3,
       0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3,
       4, 3, 1, 3, 3, 3, 0, 4, 3], dtype=int32)

In [21]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
print(df_torfin.shape)

toronto_merged = df_torfin

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')
toronto_merged.reset_index(drop=True, inplace=True);

print(toronto_merged.shape)
toronto_merged.head(15) # check the last columns!

(103, 5)
(101, 11)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Food & Drink Shop,Construction & Landscaping,Bus Stop,Park,College Rec Center
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Hockey Arena,Financial or Legal Service,Portuguese Restaurant,Intersection
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Bakery,Pub,Café,Park
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,Boutique
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Yoga Studio,Arts & Crafts Store,Portuguese Restaurant,Café
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,1,Pizza Place,Women's Store,Cuban Restaurant,Donut Shop,Doner Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3,Fast Food Restaurant,Women's Store,Cupcake Shop,Donut Shop,Doner Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,3,Gym,Coffee Shop,Restaurant,Japanese Restaurant,Beer Store
8,M3C,North York,Don Mills,43.725900,-79.340923,3,Gym,Coffee Shop,Restaurant,Japanese Restaurant,Beer Store
9,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,3,Pizza Place,Gastropub,Pharmacy,Athletics & Sports,Intersection


<h4> Importing Folium and plotting the clusters </h4>

In [22]:
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes  
import folium

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-3.0           |   py36h45558ae_0         640 KB  conda-forge
    folium-0.5.0               |             py

In [23]:
latitude=43.651070
longitude=-79.347015
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters